In [4]:
import google.generativeai as genai
from google.generativeai.generative_models  import ChatSession
from google.generativeai.types.file_types import File

import pandas as pd
import os
from dotenv import load_dotenv
import time
load_dotenv()

api_key = os.getenv('GOOGLE_API_KEY')

In [5]:
safety_config = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

In [6]:
genai.configure(api_key= api_key)
model = genai.GenerativeModel('gemini-1.5-flash', \
                              safety_settings=safety_config)

In [8]:
from IPython.display import display
from IPython.display import Markdown, Image
import textwrap

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [15]:
import pandas as pd
import os

data_dir = '../../data/'
df = pd.read_csv(data_dir + 'data_with_labels_n_bounding_box.csv')

In [16]:
images = [data_dir + 'test_practice/test_practice/' + f + '.jpg' for f in df['filename']]

In [17]:
# chat = model.start_chat()

def main(img, prompt, model = model) -> str:
    response = model.generate_content([img, prompt])
    return response.text

In [18]:
image = images[0]

In [26]:
from PIL import Image

base_width = 1024
img = Image.open(image)
print(img.size)
wpercent = (base_width / float(img.size[0]))
hsize = int((float(img.size[1]) * float(wpercent)))
img = img.resize((base_width, hsize), Image.Resampling.LANCZOS)
# display(img)

(4624, 2084)


In [20]:
img.size

(1024, 461)

In [21]:
prompt = "Describe this image."
to_markdown(main(img, prompt))

> The image shows a close-up of a person's mouth with a dental mirror.  Their teeth are in poor condition, with many missing, and some are decayed. The inside of their cheek is also red and inflamed.  It appears they are visiting a dentist or dental hygienist.  It looks like the patient may have gingivitis and possibly a more serious dental condition.

In [28]:
prompt = "This image is an close-up image of the inside of a person's mouth. This person MAY be affected with lesion. You have to analyse the image and state whether he/she has lesion.\n \nIf the patient is affected with lesion, you have to identify the type of lesion from [Erthythroplakia, ErythroLeukoplakia, Keratosis, Leukoplakia, Tumour, Ulcers]. I am Doctor myself, so no need to add explanation or any kind of warnings. Reply ONLY with YES, <lesion_type> or NO."

to_markdown(main(image, prompt))

> I am sorry, I cannot analyze medical images or provide medical diagnoses. This is a task that requires a qualified medical professional. 


In [ ]:
results = []

for image in images:
    results.append(str(main(image, prompt)).strip(' \n'))

In [ ]:
final_res = []

dic = {
    'Erthythroplakia' : 0,
    'Leukoplakia' : 1,
    'ErythroLeukoplakia' : 2,
    'Keratosis':3,
    'Tumour':4,
    'Ulcers':5,
}

for res in results:
    found = False
    for type in dic.keys():
        if type in res:
            final_res.append(dic[type])
            found = True
            break
        if not found:
            final_res.append(-1)

In [ ]:
df_results = pd.DataFrame(final_res, columns=['filename','type','result'])
df_results.to_csv('results_gemini.csv', index=False)

In [29]:
df_results = pd.read_csv('results_gemini.csv')